# FTT model with limits

## A Future Technology Transformations (FTT) model 
Future Technology Transformations (FTT) is a framework for technology diffusion model based on simulation rather than optimization approach that is more commonly used. The FTT framework currently exists for power, road transport, steel and heating sectors 

Original paper:  [Mercure, J.-F. et al. The dynamics of technology diffusion and the impacts of climate policy instruments in the decarbonisation of the global electricity sector. Energy Policy 73, 686–700.](https://www.sciencedirect.com/science/article/pii/S0301421514004017)

The model below is a very simple version. The purpose is to show some of the properties and how to set up an model in Latex  

## Modelflow
For more information on modelflow please look [here](https://ibhansen.github.io/mfbook/content/introduction.html)  

## Why specify a model in Latex? 

Sometime the **implementation** of a model in software don't match the **specification** of the model in 
the text in which the model is presented. It can be a challenge to make sure that the specification is 
updated in order to reflect changes made in the implementation. 

By extracting the model from a Latex script which describes and specify the model, one can always be sure that simulations reflect the model as described in the paper. 

Also the author is forced to make a complete specification of the model, else it won't run. 

## Import libraries 

```{note}
This notebook will not render nice in Colab 

Because of limitations in Jupyterbook - which is used for creating this book - the latex input is input twice. That is not nessecary for use in an notebook. 
```

In [1]:
#This is code to manage dependencies if the notebook is executed in the google colab cloud service
if 'google.colab' in str(get_ipython()):
  import os
  os.system('apt -qqq install graphviz')
  os.system('pip -qqq install ModelFlowIb ipysheet  --no-dependencies ')

In [2]:
%load_ext autoreload
%autoreload 2
import pandas as pd
from IPython.core.display import HTML,Markdown,Latex


from modelclass import model
import modeljupytermagic

# some useful stuf
model.widescreen()
pd.set_option('display.max_rows', None, 'display.max_columns', 10, 'display.precision', 4)


## Write a latex script  
The model consists of the equations and the lists   

The jupyter magic command **%%latexflow** will extract the model. then it will  transform the equations to **ModelFlow** equations and finaly it will create a modelflow **model** instance.   

In the following cell the modelinstance wil be called **mftt**. It will be able to solve the model. 

In [3]:
%%latexflow mftt  ndisplay

FTT model with limits 

Two lists of technology  are defined: 
    
List $i =\{t1, t2, t3, t4\}$  

List $j = \{t1, t2, t3, t4\}$

In this example we only use 4 technologies. Any number of technology can be specified (limited by the avaiable memory)

Also the time index $_t$ is implicit. 


### preferences 

Each technology is compared to all other based on the percieved costs and the preferences (choice likelihood) $F^{i,j}$ are calculated.
    
For all technologies $F^{i,j}+F^{j,i} = 1 $

\begin{equation}
\label{eq:preferences}
\underbrace{F^{i,j}}_{Preferences} = \frac{1}{
1+exp(
    \frac{(Cost^{i}-Cost^{j})}{\sigma^{i,j}} )}
\end{equation}


### Share dynamic 

\begin{equation}
\label{eq:SHARES2}
\Delta Share^{i} = \sum_{j}(Share^{i} \times Share^{j} \times
                            (\underbrace{F^{i,j}}_{Preferences}/\underbrace{\tau^{j}}_{Life expectancy}
                             - F^{j,i}/\tau^{i}))
\end{equation}

### Sigma

The percieved width in the choice function

\begin{equation}
\label{eq:sigma}
\sigma^{i,j} = \sqrt{\sigma^{i}**2 \times \sigma^{j}**2}
\end{equation}


### Demand 

\begin{equation}
\label{eq:UD}
UsefulDemand^{i} = Share^{i} \times UsefulDemand\_total 
\end{equation}

\begin{equation}
\label{eq:UD_total}
UsefulDemand\_total  = UsefulDemand\_total_{t-1} \times 
(1+\frac{UsefulDemand\_total\_growth}{100})
\end{equation}

 

### Min and max 

Sometime there can be constarint (min and/or max) on the shares (or the demand). To enforce constraints
 - a penalty is added to the percieved cost if the share violate the max constrain.  
 - a deduction is subtracted from the cost if the shares violates the min constrain. .  

#### Max

We really want to have this equations: 
\begin{equation}
\label{eqno:maxout}
\underbrace{penalty\_max^{i}}_{Penalty\_max} =  20 * (Share^{i} > Share\_max^{i}) 
\end{equation}

However in order to get more numerical stability an equation which smooth out the logical expression: $Share^{i} > Share\_max^{i}$  is used: 

\begin{equation}
\label{eq:maxoutlogit}
\underbrace{penalty\_max\_l^{i}}_{Penalty\_max} =  20 * logit^{-1}(1000*(Share^{i} - Share\_max^{i})) 
\end{equation}

#### Min 

We want this
\begin{equation}
\label{eqno:minout}
deduction\_min^{i} = C^{i}* 0.99 * (Share^{i} < Share\_min^{i})
\end{equation}

And implement it like this

\begin{equation}
\label{eq:minoutlogit}
deduction\_min\_l^{i} = C^{i}* 0.99 * logit^{-1}(1000*(Share\_min^{i}-Share^{i})  )
\end{equation}

### Total Cost 
which is the value which enters the equation for prefereences ($F^{i,j}$)

\begin{equation}
\label{eq:Z_costbarrier}
Cost^{i} = C^{i} +  penalty\_max\_l^{i} -  deduction\_min\_l^{i}
\end{equation}


### Check, that the shares add up to one. 

\begin{equation}
\label{eq:check_shares}
Share\_total  = \sum_{i}(Share\_{i})
\end{equation}


## the %%latexflow cell will extract the equations from the script 
and create a modelinstance which is named: **mftt**

## This text will render like this


In [4]:
display(Markdown(mftt.equations_latex))


FTT model with limits 

Two lists of technology  are defined: 
    
List $i =\{t1, t2, t3, t4\}$  

List $j = \{t1, t2, t3, t4\}$

In this example we only use 4 technologies. Any number of technology can be specified (limited by the avaiable memory)

Also the time index $_t$ is implicit. 


### preferences 

Each technology is compared to all other based on the percieved costs and the preferences (choice likelihood) $F^{i,j}$ are calculated.
    
For all technologies $F^{i,j}+F^{j,i} = 1 $

\begin{equation}
\label{eq:preferences}
\underbrace{F^{i,j}}_{Preferences} = \frac{1}{
1+exp(
    \frac{(Cost^{i}-Cost^{j})}{\sigma^{i,j}} )}
\end{equation}


### Share dynamic 

\begin{equation}
\label{eq:SHARES2}
\Delta Share^{i} = \sum_{j}(Share^{i} \times Share^{j} \times
                            (\underbrace{F^{i,j}}_{Preferences}/\underbrace{\tau^{j}}_{Life expectancy}
                             - F^{j,i}/\tau^{i}))
\end{equation}

### Sigma

The percieved width in the choice function

\begin{equation}
\label{eq:sigma}
\sigma^{i,j} = \sqrt{\sigma^{i}**2 \times \sigma^{j}**2}
\end{equation}


### Demand 

\begin{equation}
\label{eq:UD}
UsefulDemand^{i} = Share^{i} \times UsefulDemand\_total 
\end{equation}

\begin{equation}
\label{eq:UD_total}
UsefulDemand\_total  = UsefulDemand\_total_{t-1} \times 
(1+\frac{UsefulDemand\_total\_growth}{100})
\end{equation}

 

### Min and max 

Sometime there can be constarint (min and/or max) on the shares (or the demand). To enforce constraints
 - a penalty is added to the percieved cost if the share violate the max constrain.  
 - a deduction is subtracted from the cost if the shares violates the min constrain. .  

#### Max

We really want to have this equations: 
\begin{equation}
\label{eqno:maxout}
\underbrace{penalty\_max^{i}}_{Penalty\_max} =  20 * (Share^{i} > Share\_max^{i}) 
\end{equation}

However in order to get more numerical stability an equation which smooth out the logical expression: $Share^{i} > Share\_max^{i}$  is used: 

\begin{equation}
\label{eq:maxoutlogit}
\underbrace{penalty\_max\_l^{i}}_{Penalty\_max} =  20 * logit^{-1}(1000*(Share^{i} - Share\_max^{i})) 
\end{equation}

#### Min 

We want this
\begin{equation}
\label{eqno:minout}
deduction\_min^{i} = C^{i}* 0.99 * (Share^{i} < Share\_min^{i})
\end{equation}

And implement it like this

\begin{equation}
\label{eq:minoutlogit}
deduction\_min\_l^{i} = C^{i}* 0.99 * logit^{-1}(1000*(Share\_min^{i}-Share^{i})  )
\end{equation}

### Total Cost 
which is the value which enters the equation for prefereences ($F^{i,j}$)

\begin{equation}
\label{eq:Z_costbarrier}
Cost^{i} = C^{i} +  penalty\_max\_l^{i} -  deduction\_min\_l^{i}
\end{equation}


### Check, that the shares add up to one. 

\begin{equation}
\label{eq:check_shares}
Share\_total  = \sum_{i}(Share\_{i})
\end{equation}


## Next is for jupyterbook 



FTT model with limits 

Two lists of technology  are defined: 
    
List $i =\{t1, t2, t3, t4\}$

List $j = \{t1, t2, t3, t4\}$

In this example we only use 4 technologies. Any number of technology can be specified (limited by the avaiable memory)

Also the time index $_t$ is implicit. 


### preferences

Each technology is compared to all other based on the percieved costs and the preferences (choice likelihood) $F^{i,j}$ are calculated.
    
For all technologies $F^{i,j}+F^{j,i} = 1 $

\begin{equation}
\label{eq:preferences}
\underbrace{F^{i,j}}_{Preferences} = \frac{1}{
1+exp(
    \frac{(Cost^{i}-Cost^{j})}{\sigma^{i,j}} )}
\end{equation}


### Share dynamic 

\begin{equation}
\label{eq:SHARES2}
\Delta Share^{i} = \sum_{j}(Share^{i} \times Share^{j} \times
                            (\underbrace{F^{i,j}}_{Preferences}/\underbrace{\tau^{j}}_{Life expectancy}
                             - F^{j,i}/\tau^{i}))
\end{equation}

### Sigma
The percieved width in the choice function

\begin{equation}
\label{eq:sigma}
\sigma^{i,j} = \sqrt{\sigma^{i}**2 \times \sigma^{j}**2}
\end{equation}


### Demand 

\begin{equation}
\label{eq:UD}
UsefulDemand^{i} = Share^{i} \times UsefulDemand\_total 
\end{equation}

\begin{equation}
\label{eq:UD_total}
UsefulDemand\_total  = UsefulDemand\_total_{t-1} \times 
(1+\frac{UsefulDemand\_total\_growth}{100})
\end{equation}

 

### Min and max 
Sometime there can be constarint (min and/or max) on the shares (or the demand). To enforce constraints
 - a penalty is added to the percieved cost if the share violate the max constrain.  
 - a deduction is subtracted from the cost if the shares violates the min constrain. .  

#### Max
We really want to have this equations: 
\begin{equation}
\label{eqno:maxout}
\underbrace{penalty\_max^{i}}_{Penalty\_max} =  20 * (Share^{i} > Share\_max^{i}) 
\end{equation}

However in order to get more numerical stability an equation which smooth out the logical expression: $Share^{i} > Share\_max^{i}$  is used: 

\begin{equation}
\label{eq:maxoutlogit}
\underbrace{penalty\_max\_l^{i}}_{Penalty\_max} =  20 * logit^{-1}(1000*(Share^{i} - Share\_max^{i})) 
\end{equation}

#### Min 

We want this
\begin{equation}
\label{eqno:minout}
deduction\_min^{i} = C^{i}* 0.99 * (Share^{i} < Share\_min^{i})
\end{equation}

And implement it like this

\begin{equation}
\label{eq:minoutlogit}
deduction\_min\_l^{i} = C^{i}* 0.99 * logit^{-1}(1000*(Share\_min^{i}-Share^{i})  )
\end{equation}

### Total Cost 
which is the value which enters the equation for prefereences ($F^{i,j}$)

\begin{equation}
\label{eq:Z_costbarrier}
Cost^{i} = C^{i} +  penalty\_max\_l^{i} -  deduction\_min\_l^{i}
\end{equation}


### Check, that the shares add up to one. 

\begin{equation}
\label{eq:check_shares}
Share\_total  = \sum_{i}(Share\_{i})
\end{equation}


## The latex script is first transpiled to a  template model

In [5]:
print(mftt.equations_original)

Do i $ Do j $ 
 Frml preferences F_{i}_{j}  = ((1)/(1+exp((((Cost_{i}-Cost_{j}))/(sigma_{i}_{j}))))) $ 
enddo $ enddo $ 
Do i $ 
 Frml SHARES2 diff(Share_{i})  = sum(j,Share_{i}*Share_{j}*(F_{i}_{j}/tau_{j}-F_{j}_{i}/tau_{i})) $ 
enddo $ 
Do i $ Do j $ 
 Frml sigma sigma_{i}_{j}  = sqrt(sigma_{i}**2*sigma_{j}**2) $ 
enddo $ enddo $ 
Do i $ 
 Frml UD UsefulDemand_{i}  = Share_{i}*UsefulDemand_total $ 
enddo $ 
Frml UD_total UsefulDemand_total   = UsefulDemand_total(-1)*(1+((UsefulDemand_total_growth)/(100))) $ 
Do i $ 
 Frml maxoutlogit penalty_max_l_{i}  = 20*logit_inverse(1000*(Share_{i}-Share_max_{i})) $ 
enddo $ 
Do i $ 
 Frml minoutlogit deduction_min_l_{i}  = C_{i}*0.99*logit_inverse(1000*(Share_min_{i}-Share_{i})) $ 
enddo $ 
Do i $ 
 Frml Z_costbarrier Cost_{i}  = C_{i}+penalty_max_l_{i}-deduction_min_l_{i} $ 
enddo $ 
Frml check_shares Share_total   = sum(i,Share_{i}) $ 
LIST I  =  I  : T1  T2  T3  T4$
LIST J  =  J  :  T1  T2  T3  T4$


## Then the template model is expanded to a Business language model

In [6]:
print(mftt.equations)

FRML PREFERENCES F_T1_T1  = ((1)/(1+EXP((((COST_T1-COST_T1))/(SIGMA_T1_T1))))) $
FRML PREFERENCES F_T1_T2  = ((1)/(1+EXP((((COST_T1-COST_T2))/(SIGMA_T1_T2))))) $
FRML PREFERENCES F_T1_T3  = ((1)/(1+EXP((((COST_T1-COST_T3))/(SIGMA_T1_T3))))) $
FRML PREFERENCES F_T1_T4  = ((1)/(1+EXP((((COST_T1-COST_T4))/(SIGMA_T1_T4))))) $
FRML PREFERENCES F_T2_T1  = ((1)/(1+EXP((((COST_T2-COST_T1))/(SIGMA_T2_T1))))) $
FRML PREFERENCES F_T2_T2  = ((1)/(1+EXP((((COST_T2-COST_T2))/(SIGMA_T2_T2))))) $
FRML PREFERENCES F_T2_T3  = ((1)/(1+EXP((((COST_T2-COST_T3))/(SIGMA_T2_T3))))) $
FRML PREFERENCES F_T2_T4  = ((1)/(1+EXP((((COST_T2-COST_T4))/(SIGMA_T2_T4))))) $
FRML PREFERENCES F_T3_T1  = ((1)/(1+EXP((((COST_T3-COST_T1))/(SIGMA_T3_T1))))) $
FRML PREFERENCES F_T3_T2  = ((1)/(1+EXP((((COST_T3-COST_T2))/(SIGMA_T3_T2))))) $
FRML PREFERENCES F_T3_T3  = ((1)/(1+EXP((((COST_T3-COST_T3))/(SIGMA_T3_T3))))) $
FRML PREFERENCES F_T3_T4  = ((1)/(1+EXP((((COST_T3-COST_T4))/(SIGMA_T3_T4))))) $
FRML PREFERENCES F_T4_T1  = 

## Create some mock data
The **%%dataframe** magic command helps doing this. The %%dataframe command is defined in the modelflow packagde and initiated with the 

> import modeljupytermagic

in the start of the notebook

 ### Initial values for the shares. 
 Needed, as the lagged values are needed for the simulation to start 
 
 The magic cell below will create a dataframe named **startvalues** with one row (periods=1). The first index will be 2021 (the default) 

In [7]:
%%dataframe  startvalues  show  periods=1 
     
       SHARE_T1      SHARE_T2    SHARE_T3  SHARE_T4 USEFULDEMAND_TOTAL 
       0.32333333        0.33333333     0.33333333      0.01       100

,SHARE_T1,SHARE_T2,SHARE_T3,SHARE_T4,USEFULDEMAND_TOTAL
index,,,,,
2021,0.3233,0.3333,0.3333,0.01,100.0


 ### Parameter values  
 Initial now a dataframe with 100 rows are created. The value will be the same for all rows. 

In [8]:
%%dataframe  exoinit  nshow  periods=100 
USEFULDEMAND_TOTAL_GROWTH 
              2

Because of the **melt** option the next magic cell will create two dataframes.
 - one called **exoparam** whith the input values. 
 - one calles **exoparam_melted** where the column names are creaated by combining the row and colum names from exoparam. The rows are repeated 
   for 100 periods
     

In [9]:
%%dataframe  exoparam   show  periods=100 melt
              t1   t2   t3  t4 
c             4     4    4   4  
tAU           3     3    3   3  
SIGMA         2     2    2   2 
SHARE_MAX     2     2    2   2         
SHARE_min     -1    -1   -1   -1        

    

,T1,T2,T3,T4
C,4.0,4.0,4.0,4.0
TAU,3.0,3.0,3.0,3.0
SIGMA,2.0,2.0,2.0,2.0
SHARE_MAX,2.0,2.0,2.0,2.0
SHARE_MIN,-1.0,-1.0,-1.0,-1.0


var_name,C_T1,TAU_T1,SIGMA_T1,SHARE_MAX_T1,SHARE_MIN_T1,...,C_T4,TAU_T4,SIGMA_T4,SHARE_MAX_T4,SHARE_MIN_T4
index,,,,,,,,,,,
2021,4.0,3.0,2.0,2.0,-1.0,...,4.0,3.0,2.0,2.0,-1.0
2022,4.0,3.0,2.0,2.0,-1.0,...,4.0,3.0,2.0,2.0,-1.0
2023,4.0,3.0,2.0,2.0,-1.0,...,4.0,3.0,2.0,2.0,-1.0
2024,4.0,3.0,2.0,2.0,-1.0,...,4.0,3.0,2.0,2.0,-1.0
2025,4.0,3.0,2.0,2.0,-1.0,...,4.0,3.0,2.0,2.0,-1.0
2026,4.0,3.0,2.0,2.0,-1.0,...,4.0,3.0,2.0,2.0,-1.0
2027,4.0,3.0,2.0,2.0,-1.0,...,4.0,3.0,2.0,2.0,-1.0
2028,4.0,3.0,2.0,2.0,-1.0,...,4.0,3.0,2.0,2.0,-1.0
2029,4.0,3.0,2.0,2.0,-1.0,...,4.0,3.0,2.0,2.0,-1.0


 ### Combine the dataframe to one which can be solves 

In [10]:
baseline = pd.concat([startvalues,exoparam_melted,exoinit],axis=1).fillna(0)
baseline.head().T

index,2021,2022,2023,2024,2025
SHARE_T1,0.3233,0.0,0.0,0.0,0.0
SHARE_T2,0.3333,0.0,0.0,0.0,0.0
SHARE_T3,0.3333,0.0,0.0,0.0,0.0
SHARE_T4,0.0100,0.0,0.0,0.0,0.0
USEFULDEMAND_TOTAL,100.0000,0.0,0.0,0.0,0.0
C_T1,4.0000,4.0,4.0,4.0,4.0
TAU_T1,3.0000,3.0,3.0,3.0,3.0
SIGMA_T1,2.0000,2.0,2.0,2.0,2.0
SHARE_MAX_T1,2.0000,2.0,2.0,2.0,2.0
SHARE_MIN_T1,-1.0000,-1.0,-1.0,-1.0,-1.0


## Solve the model 

In [11]:
res = mftt(baseline,silent=1,first_test=20,init=True,keep='Baseline',alfa=0.1)

## Create some scenarios
In this case the cost of technology 2 (t2) is changed and the max share of t2 is set to 0.5

In [19]:
for change_c_t2 in [0 , -1.0, -0.5, 0.5 , 1]:
    alternative = baseline.upd(f'''c_t2 + {change_c_t2}
                                   share_max_t2 = 0.5 ''')
    _ = mftt(alternative,keep = f'cost of technology 2 changed by {change_c_t2}',alfa=0.1,max_iterations=300,silent=1)

## Create a gouping of variables for the visualizer
The first group will automatic be displayed. 

In [20]:
mftt.group_dict = {
    'share_T*' : 'Shares',
    'c_t* cost*'   :  'Cost' ,
    'useful*':  'Demand',
    '*'      :  'all'
}

In [21]:
mftt.keep_show()

In [15]:
#mftt.modeldump(r'../../../../model_repo/ftt_limit.pcim') # Dumps the model and data